The first part is copied from DatasetScript.py. We use it only to get a list of administered tranquilizers, and see if they are all accounted for in the conversion to a diazepam-equivalent

In [ ]:
import numpy as np
import pandas as pd
import datetime
from sklearn import preprocessing


# data
DATA_DIR = '/media/bigdata/10. Stages/3. Afgerond/2020-08 Jesse Kuiper/'
TRANQ_DIAZEPAM_FILE = 'TranquilizerDiazepamFactor.csv'
SAVE_OUTPUT = False


# selecting usefull var 
admission_columns = ["OpnameID", "PseudoID","AfdelingOmschrijving", "Opnamedatum",  "Ontslagdatum",  "Opnametijd", 
                     "Ontslagtijd",    "Spoed",  "EersteOpname",  "Geslacht", "Leeftijd_opname", 
                     "OpnamestatusOmschrijving", "Duur"]

administering_columns = ["PseudoID", "VoorschriftID", "ATC_code_omschr", "Medicijnnaam_ingevuld","Dosis", 
                         "Eenheid", "ToedienDatum", "ToedienTijd", "Toegediend", "DosisVerbruikt", 
                         "DosisOrigineel", "ToedieningIsOpgeschort", "NietToegediend"  ]

dbc_columns = [ "PseudoID", "dbcnummer","Startdatum", "Einddatum" ,"hoofddiagnose_groep", "zorgvraagzwaarte", 
               "MeervoudigeProblematiekInd", "persoonlijkheidsstoornis", "Opname", "DiagnoseDatum" ]

violent_columns = ["PseudoID", "hantering_datum", "begin_incident"]

patient_columns = ["PseudoID", "Leeftijd_startdatum_dossier" ]

def drop_by_pseudo_id(df: pd.DataFrame, pseudo_ids: list) -> pd.DataFrame:
    return df[df['PseudoID'].apply(lambda x: x not in pseudo_ids)].reset_index()

# # Load the original datasets

# load opnamens
admission = pd.read_csv(DATA_DIR + "werkbestanden-opnames/latest/werkbestand_opnames.csv", sep=';', 
                        usecols=admission_columns)


# load administered
administering = pd.read_csv(DATA_DIR + "werkbestanden-medicatie/latest/werkbestand_medicatie_toediening.csv", sep=';',
                        decimal=',', usecols=administering_columns)


# load dbc
dbc = pd.read_csv(DATA_DIR + "werkbestanden-dbc/latest/werkbestand_dbc.csv", sep=';', usecols=dbc_columns)


# load map
violent = pd.read_csv(DATA_DIR + "werkbestanden-map/latest/werkbestand_map.csv", sep=';', usecols=violent_columns)


#load patient or patient uniek
patient = pd.read_csv(DATA_DIR + "werkbestanden-patient/latest/werkbestand_patient_uniek.csv", sep=';', 
                  usecols=patient_columns)


# load conversion factors from various tranquilizers to diazepam
tranq_diazepam = {k:v for k,v in pd.read_csv(TRANQ_DIAZEPAM_FILE, sep=';')[['tranquilizer', 'factor']].values}


# # Filter datasets and fix null values

# ### Admissions

# remove incomplete admissions 
admission = admission[admission.OpnamestatusOmschrijving == "Ontslagen"]


# check for na values
assert admission.isnull().sum().sum() == 0


# change Opnamedatum Ontslagdatum to date times        
admission["OpnamedatumTijd"] = pd.to_datetime(admission["Opnamedatum"] + ' ' + admission["Opnametijd"])
admission["OntslagdatumTijd"] = pd.to_datetime(admission["Ontslagdatum"] + ' ' + admission["Ontslagtijd"])


# DateTime checks for the agression and the dbc
# these datetimes make sure it only covers the 
# If the duration of admission is less than the time check, it will take the whole admission
admission["DaysF"] = np.where(admission["Duur"]>= 14, 14, admission["Duur"])
admission["DaysP"] = np.where(admission["Duur"]>= 3, 3, admission["Duur"])

# create date time checks #these should have a max value
admission["DateTimeCheckF"] = admission["OpnamedatumTijd"] + pd.to_timedelta(admission["DaysF"], unit='d')
admission["DateTimeCheckP"] = admission["OpnamedatumTijd"] + pd.to_timedelta(admission["DaysP"], unit='d')

afd = ["Klin. Affectieve & Psychotische stoorn.","Klinische Acuut & Intensieve Zorg","Klin.Acuut & Intensieve. Zorg Jeugd", "Klin Diagn & Vroege Psychose"]

# AfdelingOmschrijving
adm_afd = pd.get_dummies(admission["AfdelingOmschrijving"])
adm_afd_sel = pd.concat([admission, adm_afd[afd]], axis=1)


del adm_afd_sel["AfdelingOmschrijving"]


# create admission1 - 8663, admission2 - 3192 and admission3 - 4685
admission1 = adm_afd_sel.copy()
admission2 = adm_afd_sel[admission["Duur"]> 14].reset_index().copy()
admission3 = adm_afd_sel[admission["Duur"]> 3].reset_index().copy()


print('All discharged admissions from the four nursing wards:', len(admission1))
print('Only admissions lasting 3 or more days:', len(admission3))
print('Only admissions lasting 14 or more days:', len(admission2))


# ### Diagnoses

# change NaN in hoofddiagnose_groep to "Lege hoofddiagnose" as this is already a variable in the table with the same meaning
dbc["hoofddiagnose_groep"] = dbc["hoofddiagnose_groep"].replace(np.nan, "Lege hoofddiagnose", regex=True)
dbc["hoofddiagnose_groep"] = dbc["hoofddiagnose_groep"].str.replace("Bijkomende codes/geen diagnose","Lege hoofddiagnose")


# create a diagnose date
# this is a limitation to be mentioned in the paper
def get_diagnosis_date(row):
    if type(row.DiagnoseDatum) == str:
        return row.DiagnoseDatum
    elif type(row.Einddatum) == str:
        return row.Einddatum
    else:
        return row.Startdatum

dbc["diagnosis_date"] = dbc.apply(lambda row: get_diagnosis_date(row), 1)
# Uncomment the following line to keep only diagnoses with a DiagnoseDatum
# dbc = dbc[dbc['DiagnoseDatum'].notnull()].reset_index(drop=True)

dbc.drop(columns=['DiagnoseDatum', 'Einddatum', 'Startdatum'], inplace=True)


# Drop rows that do not have a PseudoID, as there is no way to couple them with admissions
dbc = dbc[dbc['PseudoID'].notnull()].reset_index(drop=True)


assert dbc.isnull().sum().sum() == 0


# ### Violence incidents

# Drop rows that do not have a PseudoID, as there is no way to couple them with admissions
violent = violent[violent['PseudoID'].notnull()].reset_index(drop=True)


# change hantering_datum to date time with begin_incident
violent["hantering_datumTijd"] = pd.to_datetime(violent["hantering_datum"] + ' ' + violent["begin_incident"])


assert violent.isnull().sum().sum() == 0


# ### Patient

# Select only patients for which we also have admissions
patient = admission[['PseudoID']].merge(patient, on='PseudoID', how='left').drop_duplicates()


assert len(patient) == admission['PseudoID'].nunique()


assert patient.isnull().sum().sum() == 0


# ### Administered medication

# we are only interested in administered medicine
administering = administering[administering["Toegediend"]==1]

In [ ]:
not_in_dict = [el for el in administering['ATC_code_omschr'].unique() if el not in tranq_diazepam]
print(len(not_in_dict), 'are not in the conversion table')

Now the question is whether there are any benzodiazepines in the **big** conversion table (the original one) that are not included here. For this purpose, we must first create the big list

In [ ]:
with open('AllTranquilizers.txt') as f:
    all_tranq = [el.strip().lower() for el in f.readlines()]

In [ ]:
assert not [el for el in not_in_dict if el in all_tranq], "Some benzodiazepines were not in the conversion table"

In [ ]:
print('Everything is OK')